In [36]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))


from src.config.setup import SetupConfig
from src.data_workflow.create_tables.create_tables import retrieve_all_table_data

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read View
view_di = retrieve_all_table_data('view_data_integration')
view_di.head()

,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,FECHA_HORA_ACC,DIRECCION,GRAVEDAD,...,ESTADO,FECHA_NACIMIENTO,EDAD,GENERO,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,CLASE_DE_VEHICULO,TIPO_DE_SERVICIO,MODALIDAD,ENFUGA
0,A000037228,4399672,2014-11-27 19:00:00,15:20:00,2014,NOVIEMBRE,VIERNES,2014-11-28 10:20:00,AK 72-CL 13 02,SOLO DANOS,...,ILESO,1966-06-01 19:00:00,48.0,MASCULINO,N,1975-01-01 01:00:00,TRACTOCAMION,PUBLICO,CARGA-PESADA,N
1,A000036014,4399723,2014-11-27 19:00:00,05:10:00,2014,NOVIEMBRE,VIERNES,2014-11-28 00:10:00,AV AVENIDA BOYACA-CL 61 S 42,SOLO DANOS,...,ILESO,1979-02-04 19:00:00,35.0,MASCULINO,N,1975-01-01 01:00:00,VOLQUETA,PARTICULAR,None,N
2,A000040315,4400917,2014-12-21 19:00:00,14:30:00,2014,DICIEMBRE,LUNES,2014-12-22 09:30:00,AK 30-CL 79B 26,SOLO DANOS,...,ILESO,1988-04-20 19:00:00,26.0,MASCULINO,N,1975-01-01 01:00:00,AUTOMOVIL,PARTICULAR,None,N
3,712014100,14211,2007-01-17 19:00:00,16:00:00,2007,ENERO,JUEVES,2007-01-18 11:00:00,CL 145-114,CON HERIDOS,...,ILESO,1899-12-31 19:04:00,NaN,SIN INFORMACION,N,1975-01-01 01:00:00,None,None,None,None
4,728005900,42744,2007-10-25 19:00:00,02:00:00,2007,OCTUBRE,VIERNES,2007-10-25 21:00:00,DG 48-19A S 3,CON HERIDOS,...,ILESO,1983-09-23 19:00:00,24.0,MASCULINO,N,1975-01-01 01:00:00,BUSETA,PUBLICO,MIXTO,None


In [4]:
view_di.columns

Index(['FORMULARIO', 'CODIGO_ACCIDENTE', 'FECHA_OCURRENCIA_ACC',
       'HORA_OCURRENCIA_ACC', 'ANO_OCURRENCIA_ACC', 'MES_OCURRENCIA_ACC',
       'DIA_OCURRENCIA_ACC', 'FECHA_HORA_ACC', 'DIRECCION', 'GRAVEDAD',
       'TIPO_CAUSA', 'nroinfracciones', 'CLASE_ACC', 'LOCALIDAD', 'MUNICIPIO',
       'LATITUD', 'LONGITUD', 'CODIGO_ACCIDENTADO', 'CODIGO_VEHICULO',
       'CONDICION_ACCIDENTADO', 'ESTADO', 'FECHA_NACIMIENTO', 'EDAD', 'GENERO',
       'MUERTE_POSTERIOR', 'FECHA_POSTERIOR_MUERTE', 'CLASE_DE_VEHICULO',
       'TIPO_DE_SERVICIO', 'MODALIDAD', 'ENFUGA'],
      dtype='object')

In [18]:
# Función para convertir horas y minutos en float
def time_to_float(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_minutes = hours * 60 + minutes
    float_value = total_minutes / 60.0
    return float_value


def float_to_time(float_value):
    hours = int(float_value)
    minutes = int((float_value - hours) * 60)
    time_str = "{:02d}:{:02d}:00".format(hours, minutes)
    return time_str

def clasificar_vehiculo(clase_vehiculo):
    DOS_LLANTAS = ['BICICLETA', 'BICITAXI', 'CUATRIMOTO', 'MOTOCARRO', 'MOTOCICLETA', 'MOTOCICLO', 'TRACCION ANIMAL']
    CUATRO_LLANTAS = ['AMBULACIA', 'AUTOMOVIL', 'BUS', 'BUS ALIMENTADOR', 'BUS ARTICULADO', 'BUSETA',
                      'CAMION, FURGON', 'CAMIONETA', 'CAMPERO', 'M. AGRICOLA', 'M. INDUSTRIAL', 'METRO',
                      'MICROBUS', 'NO IDENTIFICADO', 'OTRO', 'REMOLQUE', 'TRACTOCAMION', 'TREN', 'VOLQUETA']
    if clase_vehiculo in DOS_LLANTAS:
        return 'DOS_LLANTAS'
    elif clase_vehiculo in CUATRO_LLANTAS:
        return 'CUATRO_LLANTAS'
    else:
        return 'NO_CLASIFICADO'




In [19]:
# view_di.groupby(["CLASE_DE_VEHICULO"]).size()

In [20]:
INTERES = ["HORA_OCURRENCIA_ACC", "MES_OCURRENCIA_ACC", "DIA_OCURRENCIA_ACC" , "GRAVEDAD", "LOCALIDAD", "CLASE_DE_VEHICULO", "CONDICION_ACCIDENTADO", "nroinfracciones", "TIPO_DE_SERVICIO", "GENERO", "ESTADO"]
data = view_di[INTERES]
data['HORA_OCURRENCIA_ACC_FLOAT'] = data['HORA_OCURRENCIA_ACC'].apply(time_to_float)
data['CLASIFICACION_VEHICULO'] = data['CLASE_DE_VEHICULO'].apply(clasificar_vehiculo)

data = data.drop(columns=['HORA_OCURRENCIA_ACC'])
data = data.drop(columns=['CLASE_DE_VEHICULO'])
data["nroinfracciones"] = data["nroinfracciones"].fillna(0)

In [21]:
data.head()

,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,GRAVEDAD,LOCALIDAD,CONDICION_ACCIDENTADO,nroinfracciones,TIPO_DE_SERVICIO,GENERO,ESTADO,HORA_OCURRENCIA_ACC_FLOAT,CLASIFICACION_VEHICULO
0,NOVIEMBRE,VIERNES,SOLO DANOS,FONTIBON,CONDUCTOR,1.0,PUBLICO,MASCULINO,ILESO,15.333333,CUATRO_LLANTAS
1,NOVIEMBRE,VIERNES,SOLO DANOS,CIUDAD BOLIVAR,CONDUCTOR,1.0,PARTICULAR,MASCULINO,ILESO,5.166667,CUATRO_LLANTAS
2,DICIEMBRE,LUNES,SOLO DANOS,BARRIOS UNIDOS,CONDUCTOR,0.0,PARTICULAR,MASCULINO,ILESO,14.500000,CUATRO_LLANTAS
3,ENERO,JUEVES,CON HERIDOS,SUBA,CONDUCTOR,0.0,None,SIN INFORMACION,ILESO,16.000000,NO_CLASIFICADO
4,OCTUBRE,VIERNES,CON HERIDOS,TUNJUELITO,CONDUCTOR,1.0,PUBLICO,MASCULINO,ILESO,2.000000,CUATRO_LLANTAS


In [22]:
data_encoders = data.copy()
label_encoders = {}
for column in data_encoders.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    data_encoders[column] = label_encoders[column].fit_transform(data_encoders[column])

X = data_encoders.drop(columns=['ESTADO'])
y = data_encoders['ESTADO']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
data_encoders.head()

,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,GRAVEDAD,LOCALIDAD,CONDICION_ACCIDENTADO,nroinfracciones,TIPO_DE_SERVICIO,GENERO,ESTADO,HORA_OCURRENCIA_ACC_FLOAT,CLASIFICACION_VEHICULO
0,9,6,2,7,1,1.0,4,1,1,15.333333,0
1,9,6,2,5,1,1.0,3,1,1,5.166667,0
2,2,2,2,1,1,0.0,3,1,1,14.500000,0
3,3,1,0,14,1,0.0,5,2,1,16.000000,2
4,10,6,0,17,1,1.0,4,1,1,2.000000,0


In [24]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
# Predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9423840364311633
Precision: 0.9423707640367543
Recall: 0.9423840364311633
F1 Score: 0.9423688934711016
Confusion Matrix:
[[ 66348   8414    465]
 [  8073 218900    150]
 [   338    121   1985]]


In [37]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logistic_model.predict(X_test)

# Evaluar el modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86     75227
           1       0.95      0.96      0.95    227123
           2       0.83      0.21      0.34      2444

    accuracy                           0.93    304794
   macro avg       0.88      0.68      0.72    304794
weighted avg       0.92      0.93      0.92    304794

